In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import thomsonpy.thomson_scattering.thomson_scattering_tools as thtools
import thomsonpy.data_management.formatter as fmt
import thomsonpy.data_management.fragmenter as frm
import thomsonpy.data_management.octree.octree as octr
import thomsonpy.data_management.visualizer as vis
import thomsonpy.config.paths as paths
import thomsonpy.config.solar_imager_params as sip
import thomsonpy.config.thomson_scattering_params as tsp
import thomsonpy.constants.units as units

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
prefix = "../../"
predsci_data_filepath = prefix + paths.PREDSCI_DATA_PATH + paths.PREDSCI_FILENAME
data_filepaths = [f"{prefix}{paths.OCTREE_DATA_PATH}{os.path.splitext(paths.PREDSCI_FILENAME)[0]}_{i}.data" for i in range(4)]
octree_paths = [f"{prefix}{paths.OCTREES_PATH}octree_{i}.oct" for i in range(4)]
models_path = f"{prefix}{paths.MODELS_PATH}"

print(predsci_data_filepath)
print(data_filepaths)
print(octree_paths)
print(models_path)

../../data/predictive_science/eclipse2021_mhd_final/corona/rho002.hdf
['../../data/projects/all-sun/data_format/rho002_0.data', '../../data/projects/all-sun/data_format/rho002_1.data', '../../data/projects/all-sun/data_format/rho002_2.data', '../../data/projects/all-sun/data_format/rho002_3.data']
['../../data/projects/all-sun/data_structures/octree_0.oct', '../../data/projects/all-sun/data_structures/octree_1.oct', '../../data/projects/all-sun/data_structures/octree_2.oct', '../../data/projects/all-sun/data_structures/octree_3.oct']
../../data/projects/all-sun/models/


In [ ]:
# DATA FRAGMENTING
print("\nStarting data formatting and fragmenting...")

ne_raw = frm.get_ne_raw(predsci_data_filepath)
data_radial = frm.get_ne_raw_coords(predsci_data_filepath, "radial")
data_theta = frm.get_ne_raw_coords(predsci_data_filepath, "theta")
data_phi = frm.get_ne_raw_coords(predsci_data_filepath, "phi")

def selection1(r, theta, phi, ne):
    """
    First quadrant
    """
    cartesian_coords = fmt.spherical_to_cartesian(r, theta, phi)
    x = cartesian_coords[0]
    y = cartesian_coords[1]
    z = cartesian_coords[2]
    limit = 3
    return (x <= limit and x >= 0 and
            y <= limit and y >= 0 and
            z <= limit / 2 and z >= -limit / 2)

def selection2(r, theta, phi, ne):
    """
    Second quadrant
    """
    cartesian_coords = fmt.spherical_to_cartesian(r, theta, phi)
    x = cartesian_coords[0]
    y = cartesian_coords[1]
    z = cartesian_coords[2]
    limit = 3
    return (x <= 0 and x >= -limit and
            y <= limit and y >= 0 and
            z <= limit / 2 and z >= -limit / 2)
    
def selection3(r, theta, phi, ne):
    """
    Third quadrant
    """
    cartesian_coords = fmt.spherical_to_cartesian(r, theta, phi)
    x = cartesian_coords[0]
    y = cartesian_coords[1]
    z = cartesian_coords[2]
    limit = 3
    return (x <= 0 and x >= -limit and
            y <= 0 and y >= -limit and
            z <= limit / 2 and z >= -limit / 2)
    
def selection4(r, theta, phi, ne):
    """
    Fourth qudrant
    """
    cartesian_coords = fmt.spherical_to_cartesian(r, theta, phi)
    x = cartesian_coords[0]
    y = cartesian_coords[1]
    z = cartesian_coords[2]
    limit = 3
    return (x <= limit and x >= 0 and
            y <= 0 and y >= -limit and
            z <= limit / 2 and z >= -limit / 2)
    
for i in range(0, 4):
    
    selection = None
    if i == 0:
        selection = selection1
    elif i == 1:
        selection = selection2
    elif i == 2:
        selection = selection3
    elif i == 3:
        selection = selection4
        
    ini_time = time.perf_counter()
    octree_data = []
    octree_data = frm.fragment(selection, fmt.apply_octree_data_format, ne_raw, data_radial, data_theta, data_phi)
    fin_time = time.perf_counter()

    print("Data formatting and fragmentation in", fin_time - ini_time, "seconds.")

    fmt.dump(data_filepaths[i], octree_data)
    print(f"Stored data for octree at {paths.OCTREE_DATA_PATH}")

    print("Octree will have", len(octree_data), "points")

In [15]:
vis.vis_octree_data(data_filepaths[0])
vis.vis_octree_data(data_filepaths[1])
vis.vis_octree_data(data_filepaths[2])
vis.vis_octree_data(data_filepaths[3])

In [ ]:
import os
import time

import thomsonpy.config.octree_params as op

## OCTREE CREATION AND STORAGE
min_v = [op.MIN_1, op.MIN_2, op.MIN_3, op.MIN_4]
max_v = [op.MAX_1, op.MAX_2, op.MAX_3, op.MAX_4]

for i in range(2, 4):
    # Loading data...
    octree_data = fmt.load(data_filepaths[i])
    print("Loaded octree data:", len(octree_data), "points.")
    octree = 0
    # Creating octree...
    print("Creating octree with params:")
    print("MAX_LEVEL =", op.MAX_LEVEL)
    print("MAX_DATA =", op.MAX_DATA)
    print("MIN_V =", min_v[i])
    print("MAX_V =", max_v[i])
    ini_time = time.perf_counter()
    octree = octr.Octree(op.MAX_LEVEL, op.MAX_DATA, octree_data, min_v[i], max_v[i])
    fin_time = time.perf_counter()
    print("Octree " + str(i + 1) + " built in", str((fin_time - ini_time) / 60), "minutes.")
    octr.Octree.save(octree, octree_paths[i])
    print("Octree " + str(i + 1) + " created at", paths.OCTREES_PATH)

In [ ]:
vis.vis_octree(octr.Octree.load(octree_paths[0]))
vis.vis_octree(octr.Octree.load(octree_paths[1]))
vis.vis_octree(octr.Octree.load(octree_paths[2]))
vis.vis_octree(octr.Octree.load(octree_paths[3]))

In [3]:
"""
Starts execution of solar corona modeling with the Predictive Science model.
"""
IMAGE_NUM_POINTS = [550]#, 720, 1024]
for i in IMAGE_NUM_POINTS:
    resolution = (sip.MAX_COORD - sip.MIN_COORD) / i
    print("Imaging between", sip.IMAGE_MIN * units.METERS_TO_RSOL, "RSol and" , sip.IMAGE_MAX * units.METERS_TO_RSOL, "RSol.")
    print("Resolution = ", resolution / 1000, " km (", i, "x", i, ").")
    x_values = np.linspace(sip.MIN_COORD, sip.MAX_COORD, i) # de - a +
    #print(x_values * units.METERS_TO_RSOL)
    y_values = np.linspace(sip.MIN_COORD, sip.MAX_COORD, i)[::-1] # de + a -
    #print(y_values * units.METERS_TO_RSOL)
    model = np.zeros((i, i))
    print("# Numeric integral steps =", tsp.NUM_Z)
    #model = fmt.load(f"../../{paths.MODELS_PATH}allsun_6x6Rsol_550p_16s_7600km.np")
    model = fmt.load(f"../../{paths.MODELS_PATH}allsun_6x6Rsol_550p_16s.np")
    x_min = [i // 2, 0, 0, i // 2]
    print(x_min)
    x_max = [i, i // 2, i // 2, i]
    print(x_max)
    y_min = [0, 0, i // 2, i // 2]
    print(y_min)
    y_max = [i // 2, i // 2, i, i]
    print(y_max)

    num_points = i**2
    print(f"Num points = {num_points}")
    count = 0
    ini_time = time.perf_counter()
    for m in range(3, 4):
        print("Loading octree...")
        NE_MODEL = octr.Octree.load(octree_paths[m])
        print("Octree loaded.")
        for y in range(y_min[m], y_max[m]):
            for x in range(x_min[m], x_max[m]):
                if (x_values[x]**2 + y_values[y]**2) > tsp.SOLAR_RADIUS**2:
                    # Coordinates with the center of the Sun as Origin of the Reference System.
                    target = (x_values[x], y_values[y], 0)
                    # Creates a ThomsonGeometry object to manage the ray - tracing across the Corona.
                    TG = thtools.ThomsonGeometry(sip.SUN_CENTER, sip.OBSERVER, target, tsp.SOLAR_RADIUS)

                    # Line of sight integration generating a value for the scattered light model.
                    scattered_light = thtools.get_scattered_light(tsp.WAVELENGTH, tsp.T_SOL, tsp.X, TG.get_elongation(), tsp.INI_Z, tsp.FIN_Z, tsp.INCR_Z, TG, NE_MODEL)
                    model[y][x] = scattered_light
                else:
                    model[y][x] = 0

                # Progress
                count += 1
                if count % 10000 == 0:
                    print("Progress", count / num_points * 100,"%")
        
    fin_time = time.perf_counter()
    print("Model built in", str((fin_time - ini_time) / 60 / 60), "hours.")

    fmt.dump(f"../../{paths.MODELS_PATH}allsun_6x6Rsol_{i}p_16s.np", model)

Imaging between [-3. -3.] RSol and [3. 3.] RSol.
Resolution =  7596.436363636363  km ( 550 x 550 ).
# Numeric integral steps = 16.0
[275, 0, 0, 275]
[550, 275, 275, 550]
[0, 0, 275, 275]
[275, 275, 550, 550]
Num points = 302500
Loading octree...
Octree loaded.
Progress 3.3057851239669422 %
Progress 6.6115702479338845 %
Progress 9.917355371900827 %
Progress 13.223140495867769 %
Progress 16.528925619834713 %
Progress 19.834710743801654 %
Progress 23.140495867768596 %
Model built in 0.7299557530833334 hours.


In [7]:
fmt.dump(f"../../{paths.MODELS_PATH}allsun_6x6Rsol_{i}p_16s.np", model)